In [1]:
from baseline import transform_test_data
from eval import evaluate_clusters_iou, count_matches, evaluate_clusters_entropy
from api_utils import run_inference
import json
import numpy as np

In [2]:
%env TOGETHER_API_KEY=86b024a1d75b8017541ed145fbeffbdf07aff06e41c5e977a36987ee6825072b

env: TOGETHER_API_KEY=86b024a1d75b8017541ed145fbeffbdf07aff06e41c5e977a36987ee6825072b


In [4]:
with open('data/splits/connections_train.json', 'r') as file:
    data = json.load(file)

In [5]:
test_data = transform_test_data(data, False)
prompt = """Given 16 items, find groups of four items that share something in common.

Category Examples
Fish: BASS, FLOUNDER, SALMON, TROUT
Fire ___: ANT, DRILL, ISLAND, OPAL
Categories will always be more specific than “5-letter-words,” “Names” or “Verbs.”
Each puzzle has exactly one solution. Watch out for words that seem to belong to multiple categories!
The following are three examples of 16 words and their corresponding answers. You are also provided a brief description for what each group shares. Learn from these examples and respond with just the answers for the following puzzle. Respond in the same format as the examples: 4 lines, with a group on each line. Follow the restrictions given at the end of the prompt. The order within a group and between groups doesn't matter, but the groups in the example answers and descriptions are given in the order of easiest to trickiest to solve.

Examples:

Example 1. HYPE, HITCH, AMP, ACOUSTIC, GAS, CABLE, ELECTRIC, SONIC, LINK, FIRE, WATER, COUPLE, HEARD, PUMP, AUDITORY, TIE

Answers: 
CABLE, ELECTRIC, GAS, WATER
ACOUSTIC, AUDITORY, HEARD, SONIC
COUPLE, HITCH, LINK, TIE
AMP, FIRE, HYPE, PUMP

Descriptions:
Monthly bills
Related to sound/hearing
Connect
Excite, with “up”

Example 2. COMPLAINT, LAWSUIT, ACTION, HANGAR, FOXGLOVE, CLAIM, RUNWAY, WINDSOCK, TERMINAL, RING, GUMSHOE, CLUB, TURNCOAT, TARMAC, BEANBAG, TORCH

Answers:
HANGAR, RUNWAY, TARMAC, TERMINAL
ACTION, CLAIM, COMPLAINT, LAWSUIT
BEANBAG, CLUB, RING, TORCH
FOXGLOVE, GUMSHOE, TURNCOAT, WINDSOCK

Descriptions:
Parts of an airport
Legal terms
Things a juggler juggles
Words ending in clothing

Example 3. FESTER, SUNDAY, FRIDAY, ROT, LURCH, CAT, CHANCE, LIP, SPOIL, THING, THURSDAY, TURN, SATURDAY, WEDNESDAY, SOUR, TUESDAY

Answers:
FRIDAY, SATURDAY, SUNDAY, THURSDAY
ROT, SOUR, SPOIL, TURN
FESTER, LURCH, THING, WEDNESDAY
CAT, CHANCE, LIP, TUESDAY

Descriptions:
Days of the week
Go bad
The Addams Family characters
Fat ___

Now answer for these 16 words. Follow these restrictions for the output:

1. DO NOT include the descriptions. DO NOT have any "Descriptions:" text.
2. ONLY include the 16 words given. There should be NO OTHER WORDS.
3. DO NOT include any preceding text, like "Answers:", or line numbers, like "1."."""

predicted_answers = await run_inference(test_data, prompt, model="databricks/dbrx-instruct")
#save the predictions in new file
with open('data/predictions/incontext_results.json', 'w') as file:
    json.dump(predicted_answers, file)

[[['NECK', 'MOON', 'RAINBOW', 'HORSESHOE'], ['HALF', 'QUARTER', 'DIME', 'PENNY'], ['IRON', 'LEAD', 'NICKEL', 'ZINC'], ['AGAIN', 'CLOVER', 'SO', 'TIN']], [['SUNDIAL', 'HOURGLASS', 'WATCH', 'CLOCK'], ['NEWTON', 'MOLE', 'PIXIE', 'J'], ['SECOND', 'HERTZ', 'NO', 'EVIL'], ['CROP', 'BOB', 'PEPPER', 'SHAG']], [['EYE', 'EAR', 'YAK', 'JAW'], ['BIB', 'GAB', 'GAG', 'POP'], ['COW', 'EWE', 'DOE', 'HEN'], ['MAR', 'SAT', 'MER', 'YAP']], [['SWALLOW', 'FISH', 'GOAT', 'TURKEY'], ['ORANGE', 'LEMON', 'DATE', 'KIWI'], ['GEORGIA', 'JORDAN', 'TOGO', 'CHAD'], ['CRANE', 'JAY', 'SCALES', 'TWINS']], [['MOOD', 'HOCKEY', 'SQUASH', 'TABLE'], ['CRICKET', 'FROG', 'HARE', 'KANGAROO'], ['BREAKING', 'CRUSH', 'MASH', 'PRESS'], ['RECORD', 'SKELETON', 'TRAMPOLINE', 'VOLLEYBALL'], [''], ['Explanation'], [''], ['MOOD', 'HOCKY', 'SQUASH', 'TABLEThesearealltypesoftablespingpong', 'airhockey', 'squash', 'andtable'], ['CRICKET', 'FROG', 'HARE', 'KANGAROOTheseareallanimals'], ['BREAKING', 'CRUSH', 'MASH', 'PRESSTheseareallverbsrel

In [7]:

ious = []
for i in range(len(test_data)):
    pred_clusters = predicted_answers[i]
    true_clusters = test_data[i]['solutions']
    print(pred_clusters)
    print(true_clusters)
    iou = evaluate_clusters_iou(pred_clusters=pred_clusters, true_clusters=true_clusters)
    ious.append(iou)
avg_iou = sum(ious) / len(ious)
print("average IOU loss in test set: ", avg_iou)

[['NECK', 'MOON', 'RAINBOW', 'HORSESHOE'], ['HALF', 'QUARTER', 'DIME', 'PENNY'], ['IRON', 'LEAD', 'NICKEL', 'ZINC'], ['AGAIN', 'CLOVER', 'SO', 'TIN']]
[['DIME', 'NICKEL', 'PENNY', 'QUARTER'], ['IRON', 'LEAD', 'TIN', 'ZINC'], ['CLOVER', 'HORSESHOE', 'MOON', 'RAINBOW'], ['AGAIN', 'HALF', 'NECK', 'SO']]
[['SUNDIAL', 'HOURGLASS', 'WATCH', 'CLOCK'], ['NEWTON', 'MOLE', 'PIXIE', 'J'], ['SECOND', 'HERTZ', 'NO', 'EVIL'], ['CROP', 'BOB', 'PEPPER', 'SHAG']]
[['CLOCK', 'HOURGLASS', 'SUNDIAL', 'WATCH'], ['BOB', 'CROP', 'PIXIE', 'SHAG'], ['HERTZ', 'MOLE', 'NEWTON', 'SECOND'], ['EVIL', 'J', 'NO', 'PEPPER']]
[['EYE', 'EAR', 'YAK', 'JAW'], ['BIB', 'GAB', 'GAG', 'POP'], ['COW', 'EWE', 'DOE', 'HEN'], ['MAR', 'SAT', 'MER', 'YAP']]
[['GAB', 'JAW', 'YAK', 'YAP'], ['COW', 'DOE', 'EWE', 'HEN'], ['BIB', 'EYE', 'GAG', 'POP'], ['EAR', 'MAR', 'MER', 'SAT']]
[['SWALLOW', 'FISH', 'GOAT', 'TURKEY'], ['ORANGE', 'LEMON', 'DATE', 'KIWI'], ['GEORGIA', 'JORDAN', 'TOGO', 'CHAD'], ['CRANE', 'JAY', 'SCALES', 'TWINS']]
[['DA

In [7]:
#read from json file
with open('data/splits/connections_train.json', 'r') as file:
    data = json.load(file)
test_data = transform_test_data(data, False)
with open('data/predictions/incontext_results.json', 'r') as file:
    predicted_answers = json.load(file)

matches = np.zeros((len(test_data),4))
entropy = np.zeros(len(test_data))

for i in range(len(test_data)):
    pred_clusters = predicted_answers[i]
    true_clusters = test_data[i]['solutions']
    print(pred_clusters)
    print(true_clusters)
    matches[i] = count_matches(pred_clusters, true_clusters)
    entropy[i] = evaluate_clusters_entropy(pred_clusters, true_clusters)

average_match_per_difficulty = np.mean(matches, axis=0)
print("average matches per difficulty: ", average_match_per_difficulty)
matches_per_puzzle = np.sum(matches, axis=1)
average_matches = np.mean(matches_per_puzzle)
print("average matches per puzzle: ", average_matches)
perfect_matches = np.sum(matches_per_puzzle == 4)
print("number of perfectly matched puzzle: ", perfect_matches, "out of ", len(test_data), "; percentage: ", perfect_matches/len(test_data) * 100, "%")
average_entropy = np.mean(entropy)
print("average cluster entropy: ", average_entropy)

[['NECK', 'MOON', 'RAINBOW', 'HORSESHOE'], ['HALF', 'QUARTER', 'DIME', 'PENNY'], ['IRON', 'LEAD', 'NICKEL', 'ZINC'], ['AGAIN', 'CLOVER', 'SO', 'TIN']]
[['DIME', 'NICKEL', 'PENNY', 'QUARTER'], ['IRON', 'LEAD', 'TIN', 'ZINC'], ['CLOVER', 'HORSESHOE', 'MOON', 'RAINBOW'], ['AGAIN', 'HALF', 'NECK', 'SO']]
[['SUNDIAL', 'HOURGLASS', 'WATCH', 'CLOCK'], ['NEWTON', 'MOLE', 'PIXIE', 'J'], ['SECOND', 'HERTZ', 'NO', 'EVIL'], ['CROP', 'BOB', 'PEPPER', 'SHAG']]
[['CLOCK', 'HOURGLASS', 'SUNDIAL', 'WATCH'], ['BOB', 'CROP', 'PIXIE', 'SHAG'], ['HERTZ', 'MOLE', 'NEWTON', 'SECOND'], ['EVIL', 'J', 'NO', 'PEPPER']]
[['EYE', 'EAR', 'YAK', 'JAW'], ['BIB', 'GAB', 'GAG', 'POP'], ['COW', 'EWE', 'DOE', 'HEN'], ['MAR', 'SAT', 'MER', 'YAP']]
[['GAB', 'JAW', 'YAK', 'YAP'], ['COW', 'DOE', 'EWE', 'HEN'], ['BIB', 'EYE', 'GAG', 'POP'], ['EAR', 'MAR', 'MER', 'SAT']]
[['SWALLOW', 'FISH', 'GOAT', 'TURKEY'], ['ORANGE', 'LEMON', 'DATE', 'KIWI'], ['GEORGIA', 'JORDAN', 'TOGO', 'CHAD'], ['CRANE', 'JAY', 'SCALES', 'TWINS']]
[['DA